# Overview

## audience
This tutorial is aimed at Python programmers who have no experience with Alexa.  I set it up for the [Santa Cruz Python Meetup](https://www.meetup.com/Santa-Cruz-Python/events/kmkhxqyzlblc/) on Aug 28, 2019.  It covers basics of how Alexa works and how to set up a functioning custom Alexa skill using AWS Lambda.  You will need a functioning Amazon developer account which you can set up at https://developer.amazon.com
as well as an AWS account which you set up at http://aws.amazon.com/

If you want to test the skill with an actual device, make sure the device is registered to the same email address you used for creating the Alexa Dev account.

## terms
To understand how Alexa works, let's first define some terms:
* Alexa device - device with microphones and Internet connection that receives verbal input
* Alexa service - Amazon endpoint that receives audio records from Alexa devices and interprets as Intents
* Intent - expression of something a user wants 
* Alexa skill - software (usually 3rd party) that provides specific functionality 
* Wake word - word that signals an Alexa device that you want to start an interaction
* Invocation - phrase that activates a specific Alexa skill
* Interaction Model - data defining a Skill's repsonses by specifying Invocation, Intents, Slots.  Saved as JSON.
* Manifest - data specifying location of an endpoint, locales, publishing information, and interfaces.  Saved as JSON.
* Reprompt - message Alexa gives after 8 seconds of silence after delivering a message expecting a user response
* Session - series of user/service interactions starting with invocation and ending when terminated by endSession response from our endpoint OR 8 seconds of silence from the user after Reprompt

## how Alexa works
Alexa devices monitor the environment for a wake word, then send the subsequent audio to the Alexa service which parses it for Intents.  If this Invokes a Custom Skill, that skill's Inteaction model will be used for defining the Intents.  That skill's manifest will specify an endpoint where the Alexa service will send a Request to handle the Intents.  The endpoint completes the loop by sending the Alexa service a Response, which the service converts to audio and plays on the device.

The Model, Requests and Responses are all JSON formatted data, defined in the Alexa documentation, specifically the [Request and Response JSON Reference](https://developer.amazon.com/docs/custom-skills/request-and-response-json-reference.html). 

## goals
In order to create a Custom Alexa skill in this tutorial, we will:
* create an Interaction Model using the Alexa Developer Console
* create code for an endpoint to respond to Requests from the Alexa Service. This code will be written in Python using the ASK SDK, and hosted on AWS Lambda.
* test these components using the Simulator in the Alexa Dev Console and with a device.

# Interaction Model
First, let's think about the kind of conversation we want.  We want Alexa to start the conversation with:
```
What is your favorite color?
```
The user would say something like:
```
it is blue
```
or
```
I like green
```
and Alexa would finish the conversation with a phrase like:
```
I like green, too.
```
or
```
I think orange is a much better color!
```

This conversation requires a single Intent to pick up the user's color.  This Intent will have sample phrases to specify how to recognize the Intent, as well as a single Slot to recognize valid words for color.

## the Developer Panel
To make the interaction model, go to the developer panel at https://developer.amazon.com/alexa/console/ask. It should look like this: ![Image](images/createSkill1.png)
If this is your first skill, no other skills will be listed yet.
You should see a blue button on the right saying `Create Skill`.  Click it to create a new skill.

## create Skill
On the next page, type the name `color fun` into the box for `Skill name` and click on the blue `Create Skill` to accept the default choices of `Custom` skill and `Provision your own`
![Image](images/createSkill2.png)


On the next page, click on the blue `Choose` button to select the default `Start from Scratch` choice
![Image](images/chooseTemplate.png)

## Build screen
This takes you to the basic build screen.  You can see on the `Skill builder checklist` panel on the right that we have automatically created an `Invocation Name` for our skill, defaulting to the 'skill name' we specified earlier ('color fun').  You can also see that we still need to create Intents and build the model.  
![Image](images/buildConsole.png)
In the navigation panel on the left, there are sections for `Invocation`, `Intents` and so on. If you wanted to change your Invocation Name, you could go to the page to do that by clicking on `Invocation`.  We want to start by creating our Intent to capture the user speaking a color. You can see that the console automatically created five Intents that Amazon requires.  We will create our own `ColorIntent` by clicking on `+ Add` next to the Intent list.

This takes you to the Add Intent page.  Type in the name of our Intent ('ColorIntent') into the box and click on the blue `Create Custom Intent` button.
![Image](images/addIntent.png)

## create Intent
This takes you to a page to set up the components of ColorIntent. 
![Image](images/sampleUtterance.png)

We want Alexa to recognize a phrase like "it is blue", so we need to add the sample phrase, "it is {slot}", where {slot} specifies the color.  We do this by starting to type "it is {" in the utterance box, which triggers a popup.
![Image](images/selectSlot1.png)

The popup gives us the opportunity to define our {color} slot.  In the `Create a new slot` box, type "color" for the slot name, and click on the `Add` button.  The utterance should be completed as "it is `{color}`" with '{color}' highlighted in color.  Hit enter to create the sample utterance.

## create Slot
This will increase the 'Sample Utterances' count to one (1), as well as create an entry below under `Intent Slots (1)` that has the name of our slot ('color') and a colored dot next to it that matches the slot highlighting above.  ![Image](images/selectSlot2.png)
In color slot row, click on the `select a slot type` drop down.  You will see a list of available slot types.  Because we haven't defined any slot types yet, this only contains "built-in" `AMAZON` slots.  You can read more about these in the [Slot Type Reference](https://developer.amazon.com/docs/custom-skills/slot-type-reference.html).

We could create our own slot type with a specific list of 'colors', but we will use the built-in `AMAZON.Color`.  Select that from the drop down, then return to the `Sample Utterances` (What might a user say to invoke this intent?) box, and type other phrases like "I like {color}" (to capture our target "I like green" phrase). Note that the console helps with autocompletion of slot names. It also screens for non-word symbols like punctuation and numbers since these are words you want to *verbally* recognize.

## save and build
When you are done with a list of 'Sample Utterances', click on the 'Save Model' button above.  A green popup should inform you that your model was successfully saved, and remind you to "build" your model. Click on the 'Build Model' button above to do that.  You are done making your Interaction Model!  If you click on the 'Build' tab at the top to return to the Build screen, you will see that you have completed 3 out of 4 steps in the checklist.  

## setup endpoint
We will tackle the last step once we set up our endpoint for handling Alexa service requests. To prepare for that, click on the `Endpoint` option below `Intents`, `Slot types`, and `Interfaces` in the left nav panel.  You will see the endpoint setup options.
![Image](images/endpoint.png)
Click on the AWS Lambda ARN radio button. Your Skill Id will appear on the screen as well as boxes for entering the ARN for your endpoint. Leave this tab open in your browser so we can come back to it when we are [setting up our endpoint on Lambda](#setup-verification) later.

# Code

## setup
This tutorial uses the Python ASK-SDK.  The SDK code is available at https://github.com/alexa/alexa-skills-kit-sdk-for-python, and the documentation is at https://alexa-skills-kit-python-sdk.readthedocs.io/en/latest/legacy.html

To be able to use the library, you will need to [download it from PyPI](https://pypi.org/project/ask-sdk/).  I suggest putting them all in a folder, so you can zip them up with your source to upload to AWS Lambda. The command to download it to a `src` folder, you would execute the command:
```
pip install ask-sdk-core -t src/
```
We can see some of the libraries automatically installed, like `dateutil` and `requests`.

In [1]:
import os
os.chdir("src/")
!ls

ask_sdk_core			  dateutil
ask_sdk_core-1.11.0.dist-info	  idna
ask_sdk_model			  idna-2.8.dist-info
ask_sdk_model-1.14.0.dist-info	  lambda_function.py
ask_sdk_runtime			  __pycache__
ask_sdk_runtime-1.11.0.dist-info  python_dateutil-2.8.0.dist-info
bin				  requests
certifi				  requests-2.22.0.dist-info
certifi-2019.6.16.dist-info	  six-1.12.0.dist-info
chardet				  six.py
chardet-3.0.4.dist-info		  urllib3
colorFun.zip			  urllib3-1.25.3.dist-info


## SDK basics
The SDK simplifies creating a handler for Alexa service requests by creating the [`AbstractRequestHandler`](https://alexa-skills-kit-python-sdk.readthedocs.io/en/latest/api/core.html#ask_sdk_core.dispatch_components.request_components.AbstractRequestHandler) class.  It has two methods:
* can_handle(handler_input) - returns true or false depending on whether the Handler can handle the request in `handler_input`
* handle(handler_input) - returns the Response for a `handler_input`

To simplify creating the Response, the SDK has a [`ResponseFactory`](https://alexa-skills-kit-python-sdk.readthedocs.io/en/latest/api/core.html#ask_sdk_core.response_helper.ResponseFactory) class to help build responses via chaining. Possible responses include:
* speak("string to speak")
* ask{"string to use as reprompt")
* set_card([card](https://developer.amazon.com/docs/custom-skills/request-and-response-json-reference.html#card-object))
* set_should_end_session(boolean)

We will access the `ResponseFactory` through the [HandlerInput object](https://alexa-skills-kit-python-sdk.readthedocs.io/en/latest/api/core.html#handler-input).

Our code needs to create Handlers for the Request types we expect from the Alexa service.  These include:
* LaunchRequest - when the skill is initially invoked
* IntentRequest - when the Alexa service identifies a specific Intent from the Model
* SessionEndedRequest - when the Alexa service terminates the session

You can learn more about Request types by looking at the `request` parameter in the [request object](https://developer.amazon.com/docs/custom-skills/request-and-response-json-reference.html#request-body-parameters)

Let's look at the code.

## Imports

In [2]:
from ask_sdk_core.skill_builder import SkillBuilder
from ask_sdk_core.dispatch_components import (
    AbstractRequestHandler, AbstractExceptionHandler)
from ask_sdk_core.utils import (
    is_intent_name, is_request_type)

In [3]:
import random

## Constants

In [4]:
COLORS = ("orange", "purple", "yellow", "green")

## Request handlers

### LaunchRequest

In [ ]:
class LaunchRequestHandler(AbstractRequestHandler):
    """Handler for skill launch."""

    def can_handle(self, handler_input):
        """Can handle or not."""
        return is_request_type("LaunchRequest")(handler_input)

    def handle(self, handler_input):
        """Speak our ."""
        print("In LaunchRequestHandler")
        # we expect a response, so we need to include a reprompt with `ask`
        (handler_input.response_builder
            .speak("What is your favorite color? ")
            .ask("please tell me your favorite color. "))
        return handler_input.response_builder.response

### IntentRequest

#### ColorIntent

In [ ]:
class ColorIntentHandler(AbstractRequestHandler):
    """Handler for ColorIntent."""

    def can_handle(self, handler_input):
        """Can handle or not."""
        return (is_request_type("IntentRequest")(handler_input) and
                is_intent_name("ColorIntent")(handler_input))

    def handle(self, handler_input):
        """Compare slot value to COLORS, and generate a response."""
        print("In ColorIntentHandler")
        slot_color = handler_input.request_envelope.request.intent.slots[
            "color"].value
        if random.randrange(2) == 1:
            message = "I like " + slot_color + ", too. "
        else:
            # note: I didn't bother to check that `choice`!= `slot_color`
            message = "I think " + random.choice(COLORS) + " is a better color"
        (handler_input.response_builder
            .speak(message)
            .set_should_end_session(True))
        return handler_input.response_builder.response

#### StopIntent

In [ ]:
class StopIntentHandler(AbstractRequestHandler):
    """Handler for Cancel, Stop intents."""

    def can_handle(self, handler_input):
        """Can handle or not."""
        return (is_request_type("IntentRequest")(handler_input) and
                (is_intent_name("AMAZON.CancelIntent")(handler_input) or
                 is_intent_name("AMAZON.StopIntent")(handler_input)))

    def handle(self, handler_input):
        """Handle response."""
        print("In StopIntentHandler")
        (handler_input.response_builder
            .speak("Ok. Bye.")
            .set_should_end_session(True))
        return handler_input.response_builder.response

### SessionEndedRequest

In [ ]:
class SessionEndedRequestHandler(AbstractRequestHandler):
    """Handler for skill session end."""

    def can_handle(self, handler_input):
        """Can handle or not."""
        return is_request_type("SessionEndedRequest")(handler_input)

    def handle(self, handler_input):
        """Handle response."""
        print("In SessionEndedRequestHandler")
        print("Session ended with reason: {}".format(
            handler_input.request_envelope.request))
        return handler_input.response_builder.response

## Exception Handler

In [ ]:
# Exception Handler class
class CatchAllExceptionHandler(AbstractExceptionHandler):
    """Catch All Exception handler.

    This handler catches all kinds of exceptions and prints
    the stack trace on AWS Cloudwatch with the request envelope.
    """

    def can_handle(self, handler_input, exception):
        """Can handle or not."""
        return True

    def handle(self, handler_input, exception):
        """Handle response."""
        # type: (HandlerInput, Exception) -> Response
        print(exception, exc_info=True)
        print("Original request was {}".format(
            handler_input.request_envelope.request))
        speech = "Sorry, there was some problem. Please try again!!"
        handler_input.response_builder.speak(speech).ask(speech)
        return handler_input.response_builder.response


## SkillBuilder
We put all these together in a SkillBuilder object we can give to AWS Lambda for producing our responses.

In [ ]:
sb = SkillBuilder()

# Add all request handlers to the skill.
sb.add_request_handler(LaunchRequestHandler())
sb.add_request_handler(ColorIntentHandler())
sb.add_request_handler(StopIntentHandler())
sb.add_request_handler(SessionEndedRequestHandler())

# Add exception handler to the skill.
sb.add_exception_handler(CatchAllExceptionHandler())

# Expose the lambda handler to register in AWS Lambda.
lambda_handler = sb.lambda_handler()

You will need to take all the code above and paste it (or your own variation) into a file called `lambda_function.py`.  Put this in your `src` directory with all the `ask-sdk-core` modules.

You can also take the `lambda_function.py` (and other module files) from the [GitHub repo](https://github.com/ultradian/alexa-color-skill/tree/master/src).


## zip code
`cd` to the `src` directory you created which should contain the `ask-sdk-core` modules and your `lambda_function.py`.  zip all this up with a zip command like:
```
zip -r colorFun.zip * -x colorFun.zip
```
Next, we will take the zip file we created here, and upload it to AWS Lambda.

# Lambda

Open your AWS lambda console at: https://console.aws.amazon.com/lambda/.  It looks like this:![Image](images/lambdaConsole.png)
Click on the orange `Create function` button on the upper right to make a new function.  

## create function
On this page, select `Author from scratch`, and under `Basic Information` enter a function name like `colorFun`.  In the `runtime` drop down, select `Python 3.6`.

## setup permissions
In the `Permissions` section, click on the triangle to expand `Choose or create an execution role`. It should show
...
It is creating an AWS IAM role we can use to give our Lambda function access to other AWS services like CloudWatch, where our logs will be written.

Click on the orange `Create function` button on the bottom right to really create the new function.  You will see the page ![Image](images/createFunction.png)

## setup trigger
The top section is `Designer` whic graphically shows that our `colorFun` function can output to Amazon CloudWatch Logs.  In order to have it triggered by the Alexa service, click on `+ Add trigger` on the left.  From the dropdown under `Trigger configuration`, select `Alexa Skills Kit` ![Image](images/lambdaTrigger.png)

## setup verification
This takes you to a new screen where you can specify the Skill ID to get greater security, so your endpoint will only respond to Requests from YOUR specific Skill.  ![Image](images/skillIDverification.png) To get the Skill ID, go back to the Build tab of the Developer Console where we [started setting up our endpoint](#setup-endpoint) for the Alexa service.  Click where it says, `Copy to Clipboard` under `Your Skill ID`.  Then, return to the Lambda console and paste the value into the `Skill ID` box and click on the orange `Add` button on the bottom right.

To tell the Alexa service where your endpoint is, go to the upper right corner of your Lambda console where your ***ARN*** is printed. It should start with 'arn:aws:lambda' and end with 'colorFun'.  There is a 'copy' icon to the right of it which you can click on to copy the value. Go back to the Endpoint screen of the Alexa dev panel and paste the value into the `Default Region` box. Click on the `Save Endpoints` button on the top. The Alexa service will run a quick check to make sure all the IDs match up, then should give you a 'successful Manifest update' message.

## upload code
Returning to the Lambda console, click on the `colorFun` function icon in the `Designer` panel.  It should bring up the `Function code` panel below.  Select `Upload a zip file` from the `Code entry type` drop down as show below. ![Image](images/codeUpload.png) Click on the `Upload` button. Select the `colorFun.zip` file we [created before](#zip-code). 

## enable testing
Next we need to enable testing.  Go to the Test tab in the Alexa Dev console. Right now, it should look like ![Image](images/enableTest.png) In the `Test is disabled for this skill` drop down at the top, select `Development`. This should enable all the parts of the screen.

## simulator testing
To test with the simulator, invoke your skill by typing the phrase you want to say in the `type or click and hold the mic` box.  Namely, "Alexa, open color fun". You should get a audio response from Alexa and a typed response like: ![Image](images/invokeSkill.png)

The enabling process should also make your skill function on all your registered devices. No one else will be able to use your skill unless you create a 'beta test' under the `Distribution` tab by completing all the sections there and sending your test user an activation email. No one in the general public would be able to use your skill until you complete the certification process that is initiated by completing all the steps under the `Certification` tab. 

You are done with this tutorial. Feel free to suggest corrections or ask questions through the [github repo](https://github.com/ultradian/alexa-color-skill) where this document lives. 

[![Creative Commons License](https://i.creativecommons.org/l/by-sa/4.0/88x31.png)](http://creativecommons.org/licenses/by-sa/4.0/) The content of this notebook is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/).

The Python code is licensed under MIT.